In [1]:
import pandas as pd
import random
import os
import sys
import requests 
from tqdm import tqdm 
import atexit
import numpy as np

In [ ]:
IN_STEM = "/shared/4/projects/podcasts/mp3Links/transcriptLinks/"
IN_PATH = IN_STEM + sys.argv[1]

OUT_STEM = "/shared/4/projects/podcasts/mp3Links/transcriptTexts/" 
OUT_PATH = OUT_STEM + sys.argv[1].replace("Links", "Texts")

In [ ]:
df = pd.read_csv(IN_PATH, names=["index", "podUrl", "mp3Url", "transUrl", "type", "lang", "transHost"])

In [3]:
colNames = []
inDf = pd.read_csv(IN_PATH, names=["index", "podUrl", "mp3Url", "transUrl", "type", "lang", "transHost"])
toProcess = set(inDf["transUrl"])
    
processed = set([])
if os.path.exists(OUT_PATH): 
    outHandle = open(OUT_PATH, "a+")
    outDf = pd.read_csv(OUT_PATH, names=["transUrl", "text"])
    
    processed = set(outDf["transUrl"])
    
else:
    outHandle = open(OUT_PATH, "w+")
    
toProcess = toProcess - processed 
print(f"processing {len(toProcess)} records")

processing 3697 records


In [12]:
def getTranscript(inUrl): 
    try: 
        response = requests.get(inUrl)
        code = response.status_code
        text = response.text
        return [code, text]
    except: 
        print("request error")
        return [None, None]

In [13]:
def exit_handler():
    if os.path.exists(OUT_FILE): 
        outHandle.close()

atexit.register(exit_handler)

<function __main__.exit_handler()>

In [ ]:
waitTime = .1 
for transUrl in tqdm(toProcess): 
    
    #keep trying to rerequest if we get a bad code 
    while True: 
        code, text = getTranscript(transUrl)
        
        #keep requesting till we aren't being rate limited 
        if code != 429: 
            waitTime = waitTime / 10
            break 
            
        #increase wait time if being rate limited 
        waitTime = waitTime * 10
        
    if code == 200: 
        outStr = '"' + transUrl.replace('"', '') + '","' + text.replace('"', '') + '"\n'
        outHandle.write(outStr)
        
    #we still want to mark that this was processed 
    else: 
        outHandle.write('"",""\n')
    
    outHandle.flush()

  0%|          | 16/3774 [00:05<20:38,  3.03it/s]

request error


  1%|          | 46/3774 [00:13<16:31,  3.76it/s]

request error


  2%|▏         | 62/3774 [00:17<15:21,  4.03it/s]